# Training a ride duration prediction model

Entrenaremos un modelo de ML para predecir la duración de viajes en taxis

## Contexto de los datos

Los registros de viajes en taxi amarillo y verde incluyen campos que capturan fechas/horas de recogida y devolución, lugares de recogida y devolución, distancias de viaje, tarifas detalladas, tipos de tarifas, tipos de pago y recuentos de pasajeros informados por el conductor. Los datos utilizados en los conjuntos de datos adjuntos fueron recopilados y proporcionados a la Comisión de Taxis y Limusinas de la Ciudad de Nueva York (TLC) por proveedores de tecnología autorizados bajo los Programas de mejora de pasajeros de taxis y vehículos de librea (TPEP/LPEP). Los datos del viaje no fueron creados por la TLC y la TLC no garantiza la exactitud de estos datos.

### Fuente de los datos

Los datos de viajes de los taxis amarillos pueden ser descargados en `https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page`

In [2]:
import pandas as pd

In [23]:
df = pd.read_parquet('../data/green_tripdata_2024-01.parquet')
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2024-01-01 00:46:55,2024-01-01 00:58:25,N,1.0,236,239,1.0,1.98,12.8,1.0,0.5,3.61,0.0,NaN,1.0,21.66,1.0,1.0,2.75
1,2,2024-01-01 00:31:42,2024-01-01 00:52:34,N,1.0,65,170,5.0,6.54,30.3,1.0,0.5,7.11,0.0,NaN,1.0,42.66,1.0,1.0,2.75
2,2,2024-01-01 00:30:21,2024-01-01 00:49:23,N,1.0,74,262,1.0,3.08,19.8,1.0,0.5,3.00,0.0,NaN,1.0,28.05,1.0,1.0,2.75
3,1,2024-01-01 00:30:20,2024-01-01 00:42:12,N,1.0,74,116,1.0,2.40,14.2,1.0,1.5,0.00,0.0,NaN,1.0,16.70,2.0,1.0,0.00
4,2,2024-01-01 00:32:38,2024-01-01 00:43:37,N,1.0,74,243,1.0,5.14,22.6,1.0,0.5,6.28,0.0,NaN,1.0,31.38,1.0,1.0,0.00


### Descripción de las variables

* **VendorID** : ID el proveedor de TPEP que proporcionó el registro: 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.

* **lpep_pickup_datetime** : La fecha y hora en que se activó el medidor.

* **lpep_dropoff_datetime** : La fecha y hora en que se desconectó el medidor.

* **store_and_fwd_flag**: Indicadora de si el registro de viaje se mantuvo en la memoria del vehículo antes de enviarlo al proveedor, también conocido como "almacenar y reenviar", porque el vehículo no tenía una conexión con el servidor.

Al _dataset_ base, le agregamos la duración de los viajes, calculando la diferencia entre `lpep_pickup_datetime` y `lpep_dropoff_datetime`

### Feature engineering

In [24]:
# COnverting to datetime objet the start and end of
df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

In [25]:
# create target variable
df['trip_duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
# compute the minutes of durstion of each trip
df['trip_duration'] = df['trip_duration'].apply(lambda td: td.total_seconds()/60)

Hay dos tipos de viajes. ¿cuántos hay de cada tipo?

In [37]:
(
    df
    .groupby('trip_type', as_index = False)[['VendorID']]
    .count()
    .rename(columns = {"VendorID" : "total"})
)

,trip_type,total
0,1.0,51397
1,2.0,1736


¿cuánto dura en promedio el viaje por cada tipo de viaje?

## Tipo 1

In [38]:
df_ = df[df.trip_type == 1]
((df_.trip_duration >= 0) & (df_.trip_duration <= 60)).mean()

0.9911667996186548

## Tipo 2

In [39]:
df__ = df[df.trip_type == 2]
((df__.trip_duration >= 0) & (df__.trip_duration <= 60)).mean()

0.9867511520737328

In [5]:
df[['trip_duration']].head()

,trip_duration
0,0 days 00:11:30
1,0 days 00:20:52
2,0 days 00:19:02
3,0 days 00:11:52
4,0 days 00:10:59


### Distribución de la variable objetivo

In [15]:
import seaborn as sns

In [18]:
df.trip_duration.describe(percentiles = [0.05, 0.98, 0.99])

count    56551.000000
mean        18.171379
std         75.464751
min          0.000000
5%           2.883333
50%         11.400000
98%         46.816667
99%         59.825000
max       1438.983333
Name: trip_duration, dtype: float64

qué tipo de viajes duran menos de un minuto o más de una hora?

no consideraremos registros de viajes con duración atípica, para no _contaminar_ el conjunto de datos

In [13]:
# consideraremos viajes solo cuya duracion haya estado entre 1 y 60 minutos
df = df[(df.trip_duration >= 1) & (df.trip_duration <= 60)]
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,trip_duration
0,2,2024-01-01 00:46:55,2024-01-01 00:58:25,N,1.0,236,239,1.0,1.98,12.80,...,0.5,3.61,0.0,NaN,1.0,21.66,1.0,1.0,2.75,11.500000
1,2,2024-01-01 00:31:42,2024-01-01 00:52:34,N,1.0,65,170,5.0,6.54,30.30,...,0.5,7.11,0.0,NaN,1.0,42.66,1.0,1.0,2.75,20.866667
2,2,2024-01-01 00:30:21,2024-01-01 00:49:23,N,1.0,74,262,1.0,3.08,19.80,...,0.5,3.00,0.0,NaN,1.0,28.05,1.0,1.0,2.75,19.033333
3,1,2024-01-01 00:30:20,2024-01-01 00:42:12,N,1.0,74,116,1.0,2.40,14.20,...,1.5,0.00,0.0,NaN,1.0,16.70,2.0,1.0,0.00,11.866667
4,2,2024-01-01 00:32:38,2024-01-01 00:43:37,N,1.0,74,243,1.0,5.14,22.60,...,0.5,6.28,0.0,NaN,1.0,31.38,1.0,1.0,0.00,10.983333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56546,2,2024-01-31 20:46:00,2024-01-31 20:55:00,None,NaN,33,25,NaN,0.00,11.58,...,0.0,3.14,0.0,NaN,1.0,15.72,NaN,NaN,NaN,9.000000
56547,2,2024-01-31 21:06:00,2024-01-31 21:11:00,None,NaN,72,72,NaN,0.49,11.58,...,0.0,0.00,0.0,NaN,1.0,12.58,NaN,NaN,NaN,5.000000
56548,2,2024-01-31 21:36:00,2024-01-31 21:40:00,None,NaN,72,72,NaN,0.52,11.58,...,0.0,2.52,0.0,NaN,1.0,15.10,NaN,NaN,NaN,4.000000
56549,2,2024-01-31 22:45:00,2024-01-31 22:51:00,None,NaN,41,42,NaN,1.17,14.22,...,0.0,0.00,0.0,NaN,1.0,15.22,NaN,NaN,NaN,6.000000
